### Evaluating performance and drift of in hospital mortality model (xgboost) using a rolling window

In [ ]:
import datetime
import os
import pickle
import random
from datetime import date

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as st
from drift_detection.baseline_models.static.utils import run_model
from drift_detector.detector import Detector
from drift_detector.reductor import Reductor
from drift_detector.rolling_window import RollingWindow, get_label
from drift_detector.tester import TSTester
from gemini.query import get_gemini_data
from gemini.utils import import_dataset_hospital, normalize, process, scale
from matplotlib.colors import ListedColormap
from scipy.stats import pearsonr, spearmanr

from cyclops.monitor.utils import get_serving_data

In [ ]:
PATH = "/mnt/nfs/project/delirium/drift_exp/JULY-04-2022/"
TIMESTEPS = 6
AGGREGATION_TYPE = "time_flatten"
HOSPITALS = ["SMH", "MSH", "THPC", "THPM", "UHNTG", "UHNTW", "PMH", "SBK"]
OUTCOME = "mortality"
THRESHOLD = 0.05
NUM_TIMESTEPS = 6
STAT_WINDOW = 30
LOOKUP_WINDOW = 0
STRIDE = 1

SHIFT = input("Select experiment: ")
MODEL_PATH = os.path.join(PATH, "saved_models", SHIFT + "_lstm.pt")

if SHIFT == "simulated_deployment":
    exp_params = {
        "source": [datetime.date(2015, 1, 1), datetime.date(2019, 1, 1)],
        "target": [datetime.date(2019, 1, 1), datetime.date(2020, 8, 1)],
        "shift_type": "source_target",
    }

if SHIFT == "covid":
    exp_params = {
        "source": [datetime.date(2019, 1, 1), datetime.date(2020, 2, 1)],
        "target": [datetime.date(2020, 3, 1), datetime.date(2020, 8, 1)],
        "shift_type": "time",
    }

if SHIFT == "seasonal_summer":
    exp_params = {
        "source": [1, 2, 3, 4, 5, 10, 11, 12],
        "target": [6, 7, 8, 9],
        "shift_type": "month",
    }

if SHIFT == "seasonal_winter":
    exp_params = {
        "source": [3, 4, 5, 6, 7, 8, 9, 10],
        "target": [11, 12, 1, 2],
        "shift_type": "month",
    }

## Get data

In [ ]:
admin_data, x, y = get_gemini_data(PATH)

## Set constant reference distribution

In [ ]:
random.seed(1)

(X_tr, y_tr), (X_val, y_val), (X_t, y_t), feats, admin_data = import_dataset_hospital(
    admin_data,
    x,
    y,
    SHIFT,
    OUTCOME,
    HOSPITALS,
)

# Normalize data
X_tr_normalized = normalize(admin_data, X_tr, AGGREGATION_TYPE, TIMESTEPS)
X_val_normalized = normalize(admin_data, X_val, AGGREGATION_TYPE, TIMESTEPS)
X_t_normalized = normalize(admin_data, X_t, AGGREGATION_TYPE, TIMESTEPS)

if AGGREGATION_TYPE != "time":
    y_tr = get_label(admin_data, X_tr, OUTCOME)
    y_val = get_label(admin_data, X_val, OUTCOME)
    y_t = get_label(admin_data, X_t, OUTCOME)

# Scale data
X_tr_scaled = scale(X_tr_normalized)
X_val_scaled = scale(X_val_normalized)
X_t_scaled = scale(X_t_normalized)

# Process data
X_tr_final = process(X_tr_scaled, AGGREGATION_TYPE, TIMESTEPS)
X_val_final = process(X_val_scaled, AGGREGATION_TYPE, TIMESTEPS)
X_t_final = process(X_t_scaled, AGGREGATION_TYPE, TIMESTEPS)

## Create data streams

In [ ]:
START_DATE = date(2019, 1, 1)
END_DATE = date(2020, 8, 1)

In [ ]:
print("Get target data streams...")
data_streams = get_serving_data(
    x,
    y,
    admin_data,
    START_DATE,
    END_DATE,
    stride=1,
    window=1,
    encounter_id="encounter_id",
    admit_timestamp="admit_timestamp",
)

## Get prediction model

In [ ]:
MODEL_NAME = input("Select Model: ")
MODEL_PATH = PATH + "_".join([SHIFT, OUTCOME, "_".join(HOSPITALS), MODEL_NAME]) + ".pkl"
if os.path.exists(MODEL_PATH):
    optimised_model = pickle.load(open(MODEL_PATH, "rb"))
else:
    optimised_model = run_model(MODEL_NAME, X_tr_final, y_tr, X_val_final, y_val)
    pickle.dump(optimised_model, open(MODEL_PATH, "wb"))

In [ ]:
DR_TECHNIQUE = "BBSDs_untrained_FFNN"
MD_TEST = "mmd"
SAMPLE = 1000
CONTEXT_TYPE = "ffnn"
PROJ_TYPE = "ffnn"

print("Get Shift Reductor...")
reductor = Reductor(
    dr_method=DR_TECHNIQUE,
    n_features=len(feats) * TIMESTEPS,
    var_ret=0.8,
)

print("Get Shift Tester...")
tester = TSTester(tester_method=MD_TEST)

print("Get Shift Detector...")
detector = Detector(
    reductor=reductor,
    tester=tester,
    p_val_threshold=0.05,
)
detector.fit(X_val_final)

In [ ]:
print("Get Rolling Window...")

rolling_window = RollingWindow(
    admin_data=admin_data,
    shift_detector=detector,
    model=optimised_model,
)

all_runs = []
for i in range(0, 1):
    random.seed(1)
    np.random.seed(1)

    drift_metrics = rolling_window.drift(
        data_streams=data_streams,
        sample=SAMPLE,
        stat_window=STAT_WINDOW,
        lookup_window=LOOKUP_WINDOW,
        stride=STRIDE,
        model_path=MODEL_PATH,
        context_type=CONTEXT_TYPE,
        proj_type=PROJ_TYPE,
        aggregation_type=AGGREGATION_TYPE,
    )

    performance_metrics = rolling_window.performance(
        data_streams=data_streams,
        stat_window=STAT_WINDOW,
        lookup_window=LOOKUP_WINDOW,
        stride=STRIDE,
        aggregation_type=AGGREGATION_TYPE,
    )

    results = {
        "timestamps": [
            (
                datetime.datetime.strptime(date, "%Y-%m-%d")
                + datetime.timedelta(days=LOOKUP_WINDOW + STAT_WINDOW)
            ).strftime("%Y-%m-%d")
            for date in data_streams["timestamps"]
        ][:-STAT_WINDOW],
    }
    results.update(drift_metrics)
    results.update(performance_metrics)

    all_runs.append(results)

In [ ]:
avgDict = {}
for k, v in results.items():
    if not all(isinstance(s, str) for s in v):
        mean = sum(v) / float(len(v))
        ci = st.t.interval(0.95, len(v), loc=np.mean(v), scale=st.sem(v))
        avgDict[k] = [mean, ci]
avgDict

## Plot Drift and Prediction Performance

In [ ]:
threshold = 0.05
sig_drift = np.array(results["shift_detected"])[np.newaxis]

fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(6, 1, figsize=(18, 12))
cmap = ListedColormap(["lightgrey", "red"])
ax1.plot(
    results["timestamps"],
    results["p_val"],
    ".-",
    color="red",
    linewidth=0.5,
    markersize=2,
)
ax1.set_xlim(results["timestamps"][0], results["timestamps"][-1])
ax1.axhline(y=threshold, color="dimgrey", linestyle="--")
ax1.set_ylabel("P-Values", fontsize=16)
ax1.set_xticklabels([])
ax1.pcolorfast(ax1.get_xlim(), ax1.get_ylim(), sig_drift, cmap=cmap, alpha=0.4)

ax2.plot(
    results["timestamps"],
    results["distance"],
    ".-",
    color="red",
    linewidth=0.5,
    markersize=2,
)
ax2.set_xlim(results["timestamps"][0], results["timestamps"][-1])
ax2.set_ylabel("Distance", fontsize=16)
ax2.axhline(y=np.mean(results["distance"]), color="dimgrey", linestyle="--")
ax2.set_xticklabels([])
ax2.pcolorfast(ax2.get_xlim(), ax2.get_ylim(), sig_drift, cmap=cmap, alpha=0.4)

ax3.plot(
    results["timestamps"],
    results["auroc"],
    ".-",
    color="blue",
    linewidth=0.5,
    markersize=2,
)
ax3.set_xlim(results["timestamps"][0], results["timestamps"][-1])
ax3.set_ylabel("AUROC", fontsize=16)
ax3.axhline(y=np.mean(results["auroc"]), color="dimgrey", linestyle="--")
ax3.set_xticklabels([])
ax3.pcolorfast(ax3.get_xlim(), ax3.get_ylim(), sig_drift, cmap=cmap, alpha=0.4)

ax4.plot(
    results["timestamps"],
    results["auprc"],
    ".-",
    color="blue",
    linewidth=0.5,
    markersize=2,
)
ax4.set_xlim(results["timestamps"][0], results["timestamps"][-1])
ax4.set_ylabel("AUPRC", fontsize=16)
ax4.axhline(y=np.mean(results["auprc"]), color="dimgrey", linestyle="--")
ax4.set_xticklabels([])
ax4.pcolorfast(ax4.get_xlim(), ax4.get_ylim(), sig_drift, cmap=cmap, alpha=0.4)

ax5.plot(
    results["timestamps"],
    results["prec1"],
    ".-",
    color="blue",
    linewidth=0.5,
    markersize=2,
)
ax5.set_xlim(results["timestamps"][0], results["timestamps"][-1])
ax5.set_ylabel("PPV", fontsize=16)
ax5.axhline(y=np.mean(results["prec1"]), color="dimgrey", linestyle="--")
ax5.set_xticklabels([])
ax5.pcolorfast(ax5.get_xlim(), ax5.get_ylim(), sig_drift, cmap=cmap, alpha=0.4)

ax6.plot(
    results["timestamps"],
    results["rec1"],
    ".-",
    color="blue",
    linewidth=0.5,
    markersize=2,
)
ax6.set_xlim(results["timestamps"][0], results["timestamps"][-1])
ax6.set_ylabel("Sensitivity", fontsize=16)
ax6.set_xlabel("time (s)", fontsize=16)
ax6.axhline(y=np.mean(results["rec1"]), color="dimgrey", linestyle="--")
ax6.tick_params(axis="x", labelrotation=45)
ax6.pcolorfast(ax6.get_xlim(), ax6.get_ylim(), sig_drift, cmap=cmap, alpha=0.4)

for index, label in enumerate(ax6.xaxis.get_ticklabels()):
    if index % 28 != 0:
        label.set_visible(False)

plt.show()

## Retraining: Drift Alarms 

### Drift Alarms

In [ ]:
baseline = [127, 118, 119, 123, 127]
mostrecent30 = [132, 116, 97, 98, 128]
mostrecent60 = [100, 96, 108, 97, 97]
mostrecent120 = [96, 76, 101, 67, 89]
cumulative = [72, 112, 64, 85, 107]

retraining_drift = pd.DataFrame(
    {
        "Baseline": baseline,
        "Most Recent \n(30 days)": mostrecent30,
        "Most Recent \n(60 days)": mostrecent60,
        "Most Recent \n(120 days)": mostrecent120,
        "Cumulative": cumulative,
    },
)
fig, ax = plt.subplots(figsize=(7, 4))
ax.boxplot(retraining_drift, patch_artist=True)
ax.set_xticks([1, 2, 3, 4, 5], retraining_drift.columns, rotation=45, fontsize=12)
ax.set_xlabel("Retraining Strategies", fontsize=12)
ax.set_ylabel("Number of Drift Alarms", fontsize=12)
plt.show()

### Number of Epochs

In [ ]:
baseline = [127, 118, 119, 123, 127]
mostrecent120 = [96, 76, 101, 67, 89]
mostrecent120_10 = [97, 103, 98, 64, 94]

retraining_drift = pd.DataFrame(
    {
        "Baseline": baseline,
        "Most Recent \n(120 days, 1 epoch)": mostrecent120,
        "Most Recent\n (120 days, 10 epochs)": mostrecent120_10,
    },
)
fig, ax = plt.subplots(figsize=(7, 4))
ax.boxplot(retraining_drift, patch_artist=True)
ax.set_xticks([1, 2, 3], retraining_drift.columns, rotation=45, fontsize=12)
ax.set_xlabel("Retraining Strategies", fontsize=12)
ax.set_ylabel("Number of Drift Alarms", fontsize=12)
plt.show()

### Drift Threshold

In [ ]:
mostrecent120_10_2 = [50, 44, 40, 51, 61]
mostrecent120 = [96, 76, 101, 67, 89]
mostrecent120_10_1 = [121, 150, 123, 139, 131]

retraining_drift = pd.DataFrame(
    {
        "P-Val=0.01": mostrecent120_10_2,
        "P-Val=0.05": mostrecent120,
        "P-Val=0.1": mostrecent120_10_1,
    },
)
fig, ax = plt.subplots(figsize=(7, 4))
ax.boxplot(retraining_drift, patch_artist=True)
ax.set_xticks([1, 2, 3], retraining_drift.columns, rotation=45, fontsize=12)
ax.set_xlabel("Retraining Strategies", fontsize=12)
ax.set_ylabel("Number of Drift Alarms", fontsize=12)
plt.show()

## Retraining: PPV & Sensitivity

### Window Size

In [ ]:
types = [
    "baseline",
    "mostrecent30",
    "mostrecent60",
    "mostrecent120",
    "cumulative_1epoch",
]
labels = [
    "Baseline",
    "Most Recent\n(30days)",
    "Most Recent\n(60 days)",
    "Most Recent\n(120 days)",
    "Cumulative",
]

drift_sensitivity = []
drift_ppv = []
for retraining_type in types:
    for i in range(0, 5):
        res_path = os.path.join(
            PATH,
            SHIFT,
            SHIFT + "_" + retraining_type + "_retraining_update.npy",
        )
        cum = np.load(res_path, allow_pickle=True)[i]
        drift_sensitivity.append(np.mean(cum["performance"]["rec1"]))
        drift_ppv.append(np.mean(cum["performance"]["prec1"]))
        # drift_sensitivity.append(np.mean(cum['performance']
        # ['rec1'][[i for i,v in enumerate(cum['pval']) if v < 0.05]]))
        # drift_ppv.append(np.mean(cum['performance']['prec1'][
        # [i for i,v in enumerate(cum['pval']) if v < 0.05]]))

retraining_drift = pd.DataFrame(
    {
        "Retraining Strategy": np.repeat(types, 5),
        "PPV": drift_ppv,
        "Sensitivity": drift_sensitivity,
    },
)

fig, axs = plt.subplots(1, 2, figsize=(14, 4))
for j, variable in enumerate(["PPV", "Sensitivity"]):
    for i, grp in enumerate(retraining_drift.groupby("Retraining Strategy")):
        axs[j].boxplot(
            x=variable,
            data=grp[1],
            positions=[i],
            widths=0.4,
            patch_artist=True,
        )
        axs[j].set_xticks(range(0, len(types)), labels, rotation=45, fontsize=12)
        axs[j].set_ylabel(variable, fontsize=12)
plt.show()

### Number of Epochs

In [ ]:
types = ["baseline", "mostrecent120", "mostrecent120_10epochs"]
labels = ["Baseline", "Most Recent\n(120 days)", "Most Recent\n(120 days, 10 epochs)"]

drift_sensitivity = []
drift_ppv = []
for retraining_type in types:
    for i in range(0, 5):
        res_path = os.path.join(
            PATH,
            SHIFT,
            SHIFT + "_" + retraining_type + "_retraining_update.npy",
        )
        cum = np.load(res_path, allow_pickle=True)[i]
        drift_sensitivity.append(np.mean(cum["performance"]["rec1"]))
        drift_ppv.append(np.mean(cum["performance"]["prec1"]))
        # drift_sensitivity.append(np.mean(cum['performance']
        # ['rec1'][[i for i,v in enumerate(cum['pval']) if v < 0.05]]))
        # drift_ppv.append(np.mean(cum['performance']['prec1']
        # [[i for i,v in enumerate(cum['pval']) if v < 0.05]]))

retraining_drift = pd.DataFrame(
    {
        "Retraining Strategy": np.repeat(types, 5),
        "PPV": drift_ppv,
        "Sensitivity": drift_sensitivity,
    },
)

fig, axs = plt.subplots(1, 2, figsize=(14, 4))
for j, variable in enumerate(["PPV", "Sensitivity"]):
    for i, grp in enumerate(retraining_drift.groupby("Retraining Strategy")):
        axs[j].boxplot(
            x=variable,
            data=grp[1],
            positions=[i],
            widths=0.4,
            patch_artist=True,
        )
        axs[j].set_xticks(range(0, len(types)), labels, rotation=45, fontsize=12)
        axs[j].set_ylabel(variable, fontsize=12)
plt.show()

### Drift Threshold

In [ ]:
types = [
    "mostrecent120_1epoch_pval0.01",
    "mostrecent120",
    "mostrecent120_1epoch_pval0.1",
]
labels = ["P-Val=0.01", "P-Val=0.05", "P-Val=0.1"]

drift_sensitivity = []
drift_ppv = []
for retraining_type in types:
    for i in range(0, 5):
        res_path = os.path.join(
            PATH,
            SHIFT,
            SHIFT + "_" + retraining_type + "_retraining_update.npy",
        )
        cum = np.load(res_path, allow_pickle=True)[i]
        drift_sensitivity.append(np.mean(cum["performance"]["rec1"]))
        drift_ppv.append(np.mean(cum["performance"]["prec1"]))
        # drift_sensitivity.append(np.mean(cum['performance']
        # ['rec1'][[i for i,v in enumerate(cum['pval']) if v < 0.05]]))
        # drift_ppv.append(np.mean(cum['performance']['prec1']
        # [[i for i,v in enumerate(cum['pval']) if v < 0.05]]))

retraining_drift = pd.DataFrame(
    {
        "Retraining Strategy": np.repeat(types, 5),
        "PPV": drift_ppv,
        "Sensitivity": drift_sensitivity,
    },
)

fig, axs = plt.subplots(1, 2, figsize=(14, 4))
for j, variable in enumerate(["PPV", "Sensitivity"]):
    for i, grp in enumerate(retraining_drift.groupby("Retraining Strategy")):
        axs[j].boxplot(
            x=variable,
            data=grp[1],
            positions=[i],
            widths=0.4,
            patch_artist=True,
        )
        axs[j].set_xticks(range(0, len(types)), labels, rotation=45, fontsize=12)
        axs[j].set_ylabel(variable, fontsize=12)
plt.show()

## Relationship between Performance and Drift P-Value

In [ ]:
# calculate Pearson's correlation
pcorr, pcorr_pval = pearsonr(results["prec1"], results["pval"])
print("Pearsons correlation: %.3f P-Value: %.3f" % (pcorr, pcorr_pval))
# calculate spearman's correlation
scorr, scorr_pval = spearmanr(results["prec1"], results["pval"])
print("Spearmans correlation: %.3f P-Value: %.3f" % (scorr, scorr_pval))
# plot
plt.scatter(results["prec1"], results["pval"])
plt.show()

In [ ]:
# calculate Pearson's correlation
pcorr, pcorr_pval = pearsonr(results["rec1"], results["pval"])
print("Pearsons correlation: %.3f P-Value: %.3f" % (pcorr, pcorr_pval))
# calculate spearman's correlation
scorr, scorr_pval = spearmanr(results["rec1"], results["pval"])
print("Spearmans correlation: %.3f P-Value: %.3f" % (scorr, scorr_pval))
# plot
plt.scatter(results["rec1"], results["pval"])
plt.show()